In [5]:
from gensim.models import FastText
from gensim.models.fasttext import load_facebook_model

import pandas as pd
import bigjson
import numpy as np
import json
from tqdm import tqdm
import time

import tensorflow_hub as hub
import tensorflow as tf

import os

In [34]:
# Elmo related loads, inits

elmo_model = "https://tfhub.dev/google/elmo/2"

hub_layer = hub.KerasLayer(elmo_model, input_shape=[], dtype=tf.string)

In [164]:
# movie frame folders
main_path = "C:\HolisticVideoUnderstanding"

sampled_test = os.path.join(main_path, "sampled_test") # +
sampled_train = os.path.join(main_path, "sampled_train") # +
sampled_val = os.path.join(main_path, "sampled_val") # +

uniform_test = os.path.join(main_path, "uniform_test") # +
uniform_train = os.path.join(main_path, "uniform_train") # +
uniform_val = os.path.join(main_path, "uniform_val") # +

In [165]:
analyzing_group = uniform_train

print(analyzing_group)

C:\HolisticVideoUnderstanding\uniform_train


In [166]:
captions_path = r"C:/Users/PC/Desktop/FewShotPhd/data_hvu/hvu_classes/train_df.csv"
captions_df = pd.read_csv(captions_path)

captions_df.head(3)

,Tags,youtube_id,time_start,time_end,action_name,general_idx
0,carnivoran|mouth|fun|joint|human_leg|muscle|gi...,rJzHt-WU1cs,4.0,14.0,air drumming,0
1,daytime|land_vehicle|mouth|mode_of_transport|c...,l-W0altbZ3o,17.0,27.0,air drumming,1
2,mouth|fun|nose|musician|hair|keyboard_player|p...,u-nRitozDOE,1.0,11.0,air drumming,2


In [167]:
y_id = "rJzHt-WU1cs"
g_idx = 0


filter_1 = captions_df["youtube_id"] == y_id
filter_2 = captions_df["general_idx"] == g_idx

y_keywords = captions_df[filter_1 & filter_2].Tags.values.tolist()[0].split("|")
y_keywords = [keyword.replace("_", " ") for keyword in y_keywords]

##########
# !! Remove class name from these keywords
##########
print(y_keywords)

['carnivoran', 'mouth', 'fun', 'joint', 'human leg', 'muscle', 'girl', 'nose', 'diaper', 'head', 'infant', 'skin', 'neck', 'arm', 'black hair', 'forehead', 'air drumming', 'face', 'child', 'human hair color', 'thigh', 'hand', 'toddler', 'standing', 'chest', 'finger']


In [168]:
def text_feature_by_id(text):
    try:
        element_vector = np.zeros((1024,), np.float32)
        video_texts = []
        text = np.array(text)
        n_sentence = 0
        
        for token in text:
            n_sentence += 1
            video_texts.append(str(token))
            
        emb = hub_layer(tf.convert_to_tensor(video_texts))
        emb = emb.numpy().mean(axis=0).reshape(-1)
        
        return emb
    except:
        return element_vector

In [169]:
empty_list = []

for idx, class_folder in enumerate(os.listdir(analyzing_group)):
    print("Folder --> {}/{}".format(idx, len(os.listdir(analyzing_group))))

    class_name = class_folder # adjusting glasses
    class_path = os.path.join(analyzing_group,class_name)
    
    class_videos = os.listdir(class_path)
    n_class_videos = len(class_videos)
    
    for class_video in tqdm(class_videos):
        
        class_video_id = str(class_video[:class_video.rfind("_")]) # video_id
        class_gidx = int(class_video[class_video.rfind("_")+1:])
        #print(class_gidx)
        #print(class_video_id)
        #print(class_name)
        
        filter_1 = captions_df["youtube_id"] == class_video_id
        filter_2 = captions_df["general_idx"] == class_gidx

        y_keywords = captions_df[filter_1 & filter_2].Tags.values.tolist()[0].split("|")
        y_keywords = [keyword.replace("_", " ") for keyword in y_keywords]
        #print(y_keywords)
        # ! removing label from keywords if exists, because some videos dont have action name in tags...
        try:
            y_keywords.remove(class_name)
        except:
            pass
        
        #print(y_keywords)
        
        # extract features from keywords
        emb = text_feature_by_id(y_keywords)
        emb = list(emb)
            
        # save elmo features to relevant folder
        saving_path = os.path.join(class_path, class_video)
        
        if len(y_keywords) == 0:
            empty_list.append(saving_path)        
    
        np.savez(saving_path+'\Elmo_Mean.npz', Elmo=emb)

  3%|▎         | 3/100 [00:00<00:03, 27.78it/s]

Folder --> 0/64


  4%|▍         | 4/100 [00:00<00:03, 31.01it/s]

Folder --> 1/64


  4%|▍         | 4/100 [00:00<00:02, 36.36it/s]

...

  3%|▎         | 3/100 [00:00<00:03, 27.78it/s]

Folder --> 62/64


  4%|▍         | 4/100 [00:00<00:02, 33.90it/s]

Folder --> 63/64


100%|██████████| 100/100 [00:02<00:00, 35.42it/s]


In [170]:
empty_list

['C:\\HolisticVideoUnderstanding\\uniform_train\\calf roping\\FelgRFIXe5g_1012',
 'C:\\HolisticVideoUnderstanding\\uniform_train\\golf putting\\cF-GGUTzORE_2975',
 'C:\\HolisticVideoUnderstanding\\uniform_train\\milking cow\\GUP-Kg1KG74_3857']

#### Val --> Empty Set

#### Test Set -->
['C:\\HolisticVideoUnderstanding\\sampled_test\\shearing sheep\\8dDUvf4-cx8_2545',
 'C:\\HolisticVideoUnderstanding\\sampled_test\\shearing sheep\\CmKsFNJfH1s_2513',
 'C:\\HolisticVideoUnderstanding\\sampled_test\\using circular saw\\fRKePKEdUSA_3096']

#### Train Set -->
['C:\\HolisticVideoUnderstanding\\sampled_train\\calf roping\\FelgRFIXe5g_1012',
 'C:\\HolisticVideoUnderstanding\\sampled_train\\golf putting\\cF-GGUTzORE_2975',
 'C:\\HolisticVideoUnderstanding\\sampled_train\\milking cow\\GUP-Kg1KG74_3857']